In [29]:
#imports
import requests
import pandas as pd
import json
import time

In [30]:
class Release_Node:
    """
    Represents a node in a networkx diagram.with basic attributes and functionalities(TBD).

    Attributes:
        release_name (str): Name of the music release.
        release_mbid (str): MusicBrainz ID.
        release_credits (list): List of credits on the music release.
    """
    def __init__(self, release_name, release_data, release_credits):
        """
        Initializes the Release_Node object.

        Args:
            release_name (str): Name of the music release.
            release_mbid (str): MusicBrainz ID.
            release_credits (list): List of credits on the music release.
        """
        self.release_name = name
        self.release_mbid = data
        self.release_credits = []

In [31]:
def search_recording(artist_name, release_title):
    """
    Retrieves all release groups for a given artist using their MusicBrainz ID.

    Args:
        artist_name (str): Name of the artist.
        release_title (str): Name of the Release

    Returns:
        list: The MBID for the release. This is the initial node for the network.
    """
    base_url = "https://musicbrainz.org/ws/2/release/"
    params = {
        "query": f"artist:\"{artist_name}\" AND release:\"{release_title}\"",
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"} # Replace with your app info

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status() # Raise an exception for bad status codes
    data = response.json()
    
    if data and data.get('releases'):
        # Return the MBID of the first matching recording
        return data['releases'][0]['id']
    return None

# Example usage:
artist = "Steely Dan"
album = "The Royal Scam"
release_mbid = search_recording(artist, album)
print(f"Recording MBID for '{album}' by '{artist}': {release_mbid}")

Recording MBID for 'The Royal Scam' by 'Steely Dan': d9b38837-6cc7-4b33-81ef-192634261abc


In [32]:
async def get_credits_for_recording(release_mbid):
    """
    Retrieves all the credits for a given release using the releases MusicBrainz ID.

    Args:
        release_mbid (str): MusicBrainz ID for the release.

    Returns:
        list: A list of all credited personnel (including artist) on the release based on the provided release mbid.
        This list includes the name of the credit, the role of the credit, and the mbid for the credit.
    """
    base_url = f"https://musicbrainz.org/ws/2/release/{release_mbid}"
    params = {
        "inc": "artist-credits+labels+recordings+recording-level-rels+work-rels+work-level-rels+artist-rels + release-groups",  # Include artist relationships
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"}

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status()
    data = response.json()

    credits = []
    
    if data and data.get('artist-credit'):
       for track in data['media'][0]['tracks']:
            for credit in track['recording']['relations']:
                if credit['attributes'] == []:
                    attributes = 'Not Found'
                else:
                    attributes = credit['attributes']
                credits.append((track['number'], track['title'], credit.get('artist', {}).get('name', 'Not Found'), attributes, credit.get('artist', {}).get('id', 'Not Found')))
            for credit in track['artist-credit']:
                credits.append((track['number'], track['title'], credit.get('name', 'Not Found'), credit.get('attributes', 'Not Found'), credit.get('artist', {}).get('id', 'Not Found')))
    if data['relations']:
       for credit in data['relations']:
           trackNum = 'Not Found'
           trackTitle = 'Not Found'
           if credit['attributes'] == [] or not credit['attributes']:
               attributes = 'Not Found'
           else:
               attributes = credit['attributes']
           credits.append((trackNum, trackTitle, credit.get('artist', {}).get('name', 'Not Found'), attributes, credit.get('artist', {}).get('id', 'Not Found')))

    creditNames = []
    creditList = []
    for credit in credits:
        if credit[2] not in creditNames:
            creditNames.append(credit[2])
            creditList.append((credit[2],credit[3], credit[4]))
    print(creditNames)
    print("\n")
    print(creditList)

    artist_mbid = creditList[0][2]
    print(artist_mbid)
    
    return credits

In [33]:
#Example usage:
if release_mbid:
    albumCredits = await get_credits_for_recording(release_mbid)
    #print(albumCredits)
else:
    print(f"Recording for '{album}' by '{artist}' not found.")

time.sleep(2)

# Create the DataFrame
df = pd.DataFrame(albumCredits, columns = ['Track Number', 'Track Title', 'Artist', 'Role', 'Artist_MBID'])

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


# Display the DataFrame
df

['Roger Nichols', 'Elliot Scheiner', 'Walter Becker', 'Larry Carlton', 'Gary Coleman', 'Donald Fagen', 'Bob Findley', 'Chuck Findley', 'Paul Griffin', 'Don Grolnick', 'Jim Horn', 'Dick Hyde', 'Plas Johnson', 'John Klemmer', 'Bernard “Pretty” Purdie', 'Chuck Rainey', 'Barney Perkins', 'Gary Katz', 'Venetta Fields', 'Clydie King', 'Sherlie Matthews', 'Michael McDonald', 'Not Found', 'Steely Dan', 'Denny Dias', 'Elliott Randall', 'Rick Marotta', 'Dean Parks']


[('Roger Nichols', 'Not Found', 'f706ae5b-de2c-4e4a-adc2-a1b9da839d70'), ('Elliot Scheiner', 'Not Found', '83b66d72-80c6-418f-a4c3-88db9c886afe'), ('Walter Becker', ['guitar'], '8cc72457-488e-44e6-a5b3-d2da843b7245'), ('Larry Carlton', ['guitar'], 'f1f81989-dfa9-4bd3-805e-dcf3900c43e3'), ('Gary Coleman', ['percussion'], '9831e425-051c-4f5c-82c6-478dbd6265ab'), ('Donald Fagen', ['keyboard'], '70047e57-0153-4117-b0fc-a1d2e322e5ef'), ('Bob Findley', ['horn'], '9cc94beb-40f2-4b8e-9c18-feaa026f2f9a'), ('Chuck Findley', ['trumpet'], '99e

,Track Number,Track Title,Artist,Role,Artist_MBID
0,1,Kid Charlemagne,Roger Nichols,Not Found,f706ae5b-de2c-4e4a-adc2-a1b9da839d70
1,1,Kid Charlemagne,Elliot Scheiner,Not Found,83b66d72-80c6-418f-a4c3-88db9c886afe
2,1,Kid Charlemagne,Walter Becker,[guitar],8cc72457-488e-44e6-a5b3-d2da843b7245
3,1,Kid Charlemagne,Walter Becker,[bass],8cc72457-488e-44e6-a5b3-d2da843b7245
4,1,Kid Charlemagne,Larry Carlton,[guitar],f1f81989-dfa9-4bd3-805e-dcf3900c43e3
5,1,Kid Charlemagne,Larry Carlton,"[guitar, solo]",f1f81989-dfa9-4bd3-805e-dcf3900c43e3
6,1,Kid Charlemagne,Gary Coleman,[percussion],9831e425-051c-4f5c-82c6-478dbd6265ab
7,1,Kid Charlemagne,Donald Fagen,[keyboard],70047e57-0153-4117-b0fc-a1d2e322e5ef
8,1,Kid Charlemagne,Bob Findley,[horn],9cc94beb-40f2-4b8e-9c18-feaa026f2f9a
9,1,Kid Charlemagne,Chuck Findley,[trumpet],99e8f55d-7968-42cf-8fe0-e4474e1bcd33
